# ****HAMMER CANDLESTICK STRATERGY****
![HAMMER_CANDLE](https://a.c-dn.net/c/content/dam/publicsites/igcom/uk/images/ContentImage/hammer.png)

***This is a rudimentary introduction to the concept of hammercandlestick.***

*A hammer is a price pattern in candlestick charting that occurs when a security trades significantly lower than its opening, but rallies within the period to close near opening price. This pattern forms a hammer-shaped candlestick, in which the lower shadow is at least twice the size of the real body. The body of the candlestick represents the difference between the open and closing prices, while the shadow shows the high and low prices for the period.*

This algo only uses the hammer candle logic as the buying signal and holds the share for 5 time cycles and that is about it.
No other logic is used so further work is needed.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ****[MPL_Finance](https://github.com/matplotlib/mplfinance)**** 
is a visualisation tool used in finace

In [ ]:
!pip install mplfinance

# ****[Backtrader](https://www.backtrader.com/docu/quickstart/quickstart/)**** 
is used for backtrading given stratergy


In [ ]:
! pip install backtrader


In [ ]:
# Using the Cipla share for reference, any other will also do.
df= pd.read_csv('/kaggle/input/nifty50-stock-market-data/CIPLA.csv', index_col='Date')
df.index = pd.to_datetime(df.index)
df.tail()

In [ ]:
# candlestick patterns do all than make pretty
# Visualise first
import mplfinance as mpf
# only considering the recent data
recent = df[df.index > '2020-05-29']
mpf.plot(recent,type='candle')

# Hammer logic found online
 
hammer ->(((hi - lo) > 3 * (op - cl)) and((cl - lo) / (.001 + hi - lo) > 0.6) and ((op - lo) / (.001 + hi - lo) > 0.6))
# Although mentioned , i have not used the inv_hammer yet
inv_hammer ->:  (((hi - lo) > 3 * (op - cl)) and ((hi - cl) / (.001 + hi - lo) > 0.6) and ((hi - op) / (.001 + hi - lo) > 0.6))

In [ ]:
df.drop(['Symbol','Series','Prev Close','Trades','Deliverable Volume','%Deliverble','Last','VWAP','Volume','Turnover'],inplace=True,axis=1)

In [ ]:
df.head()

In [ ]:
import backtrader as bt
# Most of the code directly copied from the backtrader website********************************************
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].High
        self.dataopen = self.datas[0].Open
        self.datalow = self.datas[0].Low
        # To keep track of pending orders
        self.order = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

       
        if not self.position:
            if (((self.datahigh - self.datalow) > 3 * (self.dataopen - self.dataclose)) and((self.dataclose - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6) and ((self.dataopen - self.datalow) / (.001 + self.datahigh - self.datalow) > 0.6)):
                self.order = self.buy() 


        else:

            # Randomly selling after 3 timecyles...
            if len(self) >= (self.bar_executed + 3):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [ ]:
import backtrader as bt
cerebro = bt.Cerebro()
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
data = bt.feeds.PandasData(dataname=recent)
cerebro.adddata(data)
cerebro.addstrategy(TestStrategy)
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())